In [3]:
import requests
import pandas as pd
pd.set_option("display.max_columns", None)

df = pd.read_csv("director_info.csv")
df.head()

,name,birth_place
0,David Lynch,"Missoula, Montana, USA"
1,Wong Kar-wai,"Shanghai, China"
2,Terrence Malick,"Ottawa, Illinois, USA"
3,Edward Yang,"Shanghai, China"
4,Jean-Luc Godard,"Paris, France"


In [4]:
api_key = 'W9VVaq6bt3b-4vPHxtsrNPziDHzxrpTAHvjErbH4Gqk'

In [9]:
from tqdm.auto import tqdm
tqdm.pandas()

In [17]:
def geocode(address):
    params = {
        'q': address,
        'apikey': api_key
    }
    response = requests.get('https://geocode.search.hereapi.com/v1/geocode', params=params)
    data = response.json()
    if len(data['items']) > 0:
        match = data['items'][0]
        result = {
            'place': address,
            'lat': match['position']['lat'],
            'lng': match['position']['lng']
        }
        return pd.Series(result)
    else:
        result = {
            'place': address,
            'lat': 'NaN',
            'lng': 'NaN'
        }
        return pd.Series(result)
    
    
geocoded = df.birth_place.progress_apply(geocode)
geocoded

  0%|          | 0/412 [00:00<?, ?it/s]

,place,lat,lng
0,"Missoula, Montana, USA",46.87278,-113.99625
1,"Shanghai, China",31.25516,121.4747
2,"Ottawa, Illinois, USA",41.34656,-88.84266
3,"Shanghai, China",31.25516,121.4747
4,"Paris, France",48.85717,2.3414
...,...,...,...
407,"Bristol, England, UK",51.45379,-2.59168
408,"Uttoxeter, Staffordshire, England, UK",52.89693,-1.8698
409,"Dublin, Ireland",53.34807,-6.24827
410,"Dorset, England, UK",50.80258,-2.36075


In [18]:
len(geocoded)

412

In [38]:
merged = df.join(geocoded)
merged2 = merged.drop(columns=['place'])
merged2

,name,birth_place,lat,lng
0,David Lynch,"Missoula, Montana, USA",46.87278,-113.99625
1,Wong Kar-wai,"Shanghai, China",31.25516,121.4747
2,Terrence Malick,"Ottawa, Illinois, USA",41.34656,-88.84266
3,Edward Yang,"Shanghai, China",31.25516,121.4747
4,Jean-Luc Godard,"Paris, France",48.85717,2.3414
...,...,...,...,...
407,Christopher Morris,"Bristol, England, UK",51.45379,-2.59168
408,Shane Meadows,"Uttoxeter, Staffordshire, England, UK",52.89693,-1.8698
409,John Carney,"Dublin, Ireland",53.34807,-6.24827
410,Stephen Daldry,"Dorset, England, UK",50.80258,-2.36075


In [47]:
merged2['geometry.coordinates'] = merged2[['lng','lat']].values.tolist()
merged2

,name,birth_place,lat,lng,geometry.coordinates
0,David Lynch,"Missoula, Montana, USA",46.87278,-113.99625,"[-113.99625, 46.87278]"
1,Wong Kar-wai,"Shanghai, China",31.25516,121.4747,"[121.4747, 31.25516]"
2,Terrence Malick,"Ottawa, Illinois, USA",41.34656,-88.84266,"[-88.84266, 41.34656]"
3,Edward Yang,"Shanghai, China",31.25516,121.4747,"[121.4747, 31.25516]"
4,Jean-Luc Godard,"Paris, France",48.85717,2.3414,"[2.3414, 48.85717]"
...,...,...,...,...,...
407,Christopher Morris,"Bristol, England, UK",51.45379,-2.59168,"[-2.59168, 51.45379]"
408,Shane Meadows,"Uttoxeter, Staffordshire, England, UK",52.89693,-1.8698,"[-1.8698, 52.89693]"
409,John Carney,"Dublin, Ireland",53.34807,-6.24827,"[-6.24827, 53.34807]"
410,Stephen Daldry,"Dorset, England, UK",50.80258,-2.36075,"[-2.36075, 50.80258]"


In [48]:
merged2.to_csv("birthplace.csv",index=False)